# Switches and Event Detection

This notebook covers switch modeling and event detection in Pulsim for power electronics simulation.

## Contents
1. Switch Basics
2. Switch Parameters
3. Voltage-Controlled Switches
4. Event Detection and Bisection
5. PWM Control
6. Dead-Time Implementation
7. Synchronous Rectification

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

## 1. Switch Basics

Switches in power electronics are modeled as voltage-controlled resistors:

$$R_{switch} = \begin{cases} R_{on} & \text{if } V_{ctrl} > V_{th} \\ R_{off} & \text{if } V_{ctrl} \leq V_{th} \end{cases}$$

The switch changes state when the control voltage crosses the threshold.

In [ ]:
# Create basic switch parameters
switch = pulsim.SwitchParams()
switch.ron = 1e-3       # 1mΩ on-resistance
switch.roff = 1e9       # 1GΩ off-resistance
switch.vth = 0.5        # 0.5V threshold
switch.initial_state = False  # Start open

print("Switch Parameters:")
print(f"  Ron: {switch.ron*1000:.1f} mΩ")
print(f"  Roff: {switch.roff/1e9:.0f} GΩ")
print(f"  Threshold: {switch.vth} V")
print(f"  Initial state: {'Closed' if switch.initial_state else 'Open'}")
print(f"\n  On/Off ratio: {switch.roff/switch.ron:.0e}")

## 2. Switch Parameters

```python
SwitchParams:
    ron = 1e-3      # On-resistance (Ω)
    roff = 1e9      # Off-resistance (Ω)
    vth = 0.5       # Threshold voltage (V)
    initial_state = False  # Initial state (False=open, True=closed)
```

The switch requires control nodes to determine its state:
```python
circuit.add_switch(name, n1, n2, ctrl_pos, ctrl_neg, params)
```

Switch closes when: `V(ctrl_pos) - V(ctrl_neg) > Vth`

In [ ]:
# Simple switch demonstration
netlist = '''
{
  "name": "Basic Switch Demo",
  "components": [
    {"type": "V", "name": "Vdc", "nodes": ["vdc", "0"], "value": 12},
    {"type": "V", "name": "Vctrl", "nodes": ["ctrl", "0"],
     "waveform": {"type": "pulse", "v1": 0, "v2": 5, "td": 1e-3,
                  "tr": 1e-6, "tf": 1e-6, "pw": 2e-3, "period": 5e-3}},
    {"type": "S", "name": "S1", "nodes": ["vdc", "out", "ctrl", "0"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 2.5}},
    {"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 100}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 10e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Control voltage
axes[0].plot(time_ms, data['signals']['V(ctrl)'], 'g-', linewidth=1.5, label='V(ctrl)')
axes[0].axhline(y=2.5, color='r', linestyle='--', label='Threshold (2.5V)')
axes[0].set_ylabel('Control (V)')
axes[0].set_title('Switch Control and Output')
axes[0].legend()
axes[0].grid(True)

# Output voltage
axes[1].plot(time_ms, data['signals']['V(out)'], 'b-', linewidth=1.5)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('V(out) (V)')
axes[1].set_ylim([-0.5, 13])
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("Switch closes when V(ctrl) > 2.5V")
print("Switch opens when V(ctrl) < 2.5V")

## 3. Voltage-Controlled Switches

Switches can be controlled by any voltage in the circuit, not just dedicated sources.
This enables feedback-based control and self-oscillating circuits.

In [ ]:
# Comparator-based switch control
# Switch turns on when output drops below reference

netlist = '''
{
  "name": "Hysteretic Buck Controller",
  "components": [
    {"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": 12},
    {"type": "V", "name": "Vref", "nodes": ["ref", "0"], "value": 5},
    
    {"type": "S", "name": "S1", "nodes": ["vin", "sw", "ref", "out"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 0.1}},
    
    {"type": "D", "name": "D1", "nodes": ["0", "sw"],
     "params": {"ideal": true}},
    
    {"type": "L", "name": "L1", "nodes": ["sw", "out"], "value": 100e-6},
    {"type": "C", "name": "C1", "nodes": ["out", "0"], "value": 100e-6},
    {"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 5}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 1e-3
options.dt = 50e-9

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Switch node
axes[0].plot(time_us, data['signals']['V(sw)'], 'b-', linewidth=0.8)
axes[0].set_ylabel('V(sw) (V)')
axes[0].set_title('Hysteretic Control: Switch turns on when V(out) < V(ref)')
axes[0].grid(True)

# Output voltage with reference
axes[1].plot(time_us, data['signals']['V(out)'], 'r-', linewidth=1.5, label='V(out)')
axes[1].axhline(y=5.0, color='g', linestyle='--', label='V(ref) = 5V')
axes[1].set_xlabel('Time (µs)')
axes[1].set_ylabel('V(out) (V)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 4. Event Detection and Bisection

Pulsim uses bisection to find the exact moment when a switch changes state:

1. Detect that control voltage crossed threshold between timesteps
2. Use bisection to find exact crossing time
3. Simulate up to crossing, change switch state
4. Continue simulation

This ensures accurate timing and prevents numerical artifacts.

In [ ]:
# Demonstrate event detection with switch events
netlist = '''
{
  "name": "Event Detection Demo",
  "components": [
    {"type": "V", "name": "Vdc", "nodes": ["vdc", "0"], "value": 12},
    {"type": "V", "name": "Vpwm", "nodes": ["pwm", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 5, 
                  "frequency": 10e3, "duty": 0.5}},
    {"type": "S", "name": "S1", "nodes": ["vdc", "sw", "pwm", "0"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 2.5}},
    {"type": "R", "name": "R1", "nodes": ["sw", "0"], "value": 10}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

# Simulate with coarse timestep to show event detection
options = pulsim.SimulationOptions()
options.tstop = 500e-6  # 5 switching cycles
options.dt = 10e-6      # Coarse: 10µs (period is 100µs)

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6
v_sw = np.array(data['signals']['V(sw)'])

# Count switch events
switch_events = np.sum(np.abs(np.diff(v_sw)) > 5)  # Transitions > 5V

plt.figure(figsize=(12, 5))
plt.plot(time_us, v_sw, 'b.-', linewidth=1, markersize=3)
plt.xlabel('Time (µs)')
plt.ylabel('V(sw) (V)')
plt.title(f'Event Detection: {switch_events} switch events detected with {options.dt*1e6:.0f}µs timestep')
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"Timestep: {options.dt*1e6:.0f} µs")
print(f"Switching period: 100 µs")
print(f"Switch events detected: {switch_events}")
print(f"Total simulation steps: {result.total_steps}")

In [ ]:
# Access switch events from simulation result
if hasattr(result, 'switch_events') and result.switch_events:
    print("\nSwitch Events Log:")
    print("-" * 60)
    for event in result.switch_events[:10]:  # First 10 events
        state_str = "CLOSED" if event.new_state else "OPEN"
        print(f"  t = {event.time*1e6:.3f} µs: {event.switch_name} -> {state_str}")
        print(f"       V = {event.voltage:.2f} V, I = {event.current:.3f} A")
else:
    print("Switch events not available in result (check simulation options)")

## 5. PWM Control

PWM waveforms provide clean switch control signals with configurable:
- Frequency
- Duty cycle
- Dead-time
- Phase offset
- Complementary output

In [ ]:
# Half-bridge with PWM control
netlist = '''
{
  "name": "PWM Half-Bridge",
  "components": [
    {"type": "V", "name": "Vdc", "nodes": ["vdc", "0"], "value": 48},
    
    {"type": "V", "name": "Vpwm_h", "nodes": ["pwm_h", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 15, 
                  "frequency": 50e3, "duty": 0.45, "dead_time": 200e-9}},
    {"type": "V", "name": "Vpwm_l", "nodes": ["pwm_l", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 15, 
                  "frequency": 50e3, "duty": 0.45, "dead_time": 200e-9,
                  "complementary": true}},
    
    {"type": "S", "name": "S_high", "nodes": ["vdc", "sw", "pwm_h", "0"],
     "params": {"ron": 0.02, "roff": 1e9, "vth": 5}},
    {"type": "S", "name": "S_low", "nodes": ["sw", "0", "pwm_l", "0"],
     "params": {"ron": 0.02, "roff": 1e9, "vth": 5}},
    
    {"type": "L", "name": "Lload", "nodes": ["sw", "out"], "value": 1e-3},
    {"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 10}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 100e-6  # 5 switching cycles
options.dt = 20e-9      # Fine resolution for dead-time

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6

fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

# High-side gate
axes[0].plot(time_us, data['signals']['V(pwm_h)'], 'b-', linewidth=1)
axes[0].set_ylabel('V(pwm_h) (V)')
axes[0].set_title('PWM Half-Bridge with Dead-Time')
axes[0].set_ylim([-1, 17])
axes[0].grid(True)

# Low-side gate
axes[1].plot(time_us, data['signals']['V(pwm_l)'], 'r-', linewidth=1)
axes[1].set_ylabel('V(pwm_l) (V)')
axes[1].set_ylim([-1, 17])
axes[1].grid(True)

# Switch node
axes[2].plot(time_us, data['signals']['V(sw)'], 'g-', linewidth=1)
axes[2].set_ylabel('V(sw) (V)')
axes[2].grid(True)

# Output
axes[3].plot(time_us, data['signals']['V(out)'], 'm-', linewidth=1.5)
axes[3].set_xlabel('Time (µs)')
axes[3].set_ylabel('V(out) (V)')
axes[3].grid(True)

plt.tight_layout()
plt.show()

## 6. Dead-Time Implementation

Dead-time prevents shoot-through in half-bridge configurations:

```
High-side: ────┐     ┌────────────────┐     ┌────
               │     │                │     │
               └─────┘                └─────┘
                   ├─┤            ├─┤
                  dead            dead
                  time            time
Low-side:  ────────┐     ┌────────────────┐     
                   │     │                │     
               ────┘     └────────────────┘────
```

In [ ]:
# Zoom in on dead-time
# Select a transition region
t_start = 9.5  # µs
t_end = 11.5   # µs

mask = (time_us >= t_start) & (time_us <= t_end)
t_zoom = time_us[mask]

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Gate signals
pwm_h = np.array(data['signals']['V(pwm_h)'])[mask]
pwm_l = np.array(data['signals']['V(pwm_l)'])[mask]

axes[0].plot(t_zoom, pwm_h, 'b-', linewidth=2, label='High-side')
axes[0].plot(t_zoom, pwm_l, 'r-', linewidth=2, label='Low-side')
axes[0].axhline(y=5, color='gray', linestyle='--', alpha=0.5, label='Threshold')
axes[0].set_ylabel('Gate Voltage (V)')
axes[0].set_title('Dead-Time Detail (200ns)')
axes[0].legend()
axes[0].grid(True)

# Switch node
v_sw = np.array(data['signals']['V(sw)'])[mask]
axes[1].plot(t_zoom, v_sw, 'g-', linewidth=2)
axes[1].set_xlabel('Time (µs)')
axes[1].set_ylabel('V(sw) (V)')
axes[1].set_title('Switch Node During Dead-Time')
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("During dead-time, both switches are off.")
print("Load current freewheels through body diodes.")

In [ ]:
# Compare different dead-times
dead_times = [0, 100e-9, 500e-9, 1e-6]

fig, axes = plt.subplots(len(dead_times), 1, figsize=(12, 10), sharex=True)

for idx, dt in enumerate(dead_times):
    netlist = f'''
    {{
      "name": "Half-Bridge DT={dt*1e9:.0f}ns",
      "components": [
        {{"type": "V", "name": "Vdc", "nodes": ["vdc", "0"], "value": 48}},
        {{"type": "V", "name": "Vpwm_h", "nodes": ["pwm_h", "0"],
         "waveform": {{"type": "pwm", "v_off": 0, "v_on": 15, 
                      "frequency": 100e3, "duty": 0.5, "dead_time": {dt}}}}},
        {{"type": "V", "name": "Vpwm_l", "nodes": ["pwm_l", "0"],
         "waveform": {{"type": "pwm", "v_off": 0, "v_on": 15, 
                      "frequency": 100e3, "duty": 0.5, "dead_time": {dt},
                      "complementary": true}}}},
        {{"type": "S", "name": "S_h", "nodes": ["vdc", "sw", "pwm_h", "0"],
         "params": {{"ron": 0.01, "roff": 1e9, "vth": 5}}}},
        {{"type": "S", "name": "S_l", "nodes": ["sw", "0", "pwm_l", "0"],
         "params": {{"ron": 0.01, "roff": 1e9, "vth": 5}}}},
        {{"type": "R", "name": "R", "nodes": ["sw", "0"], "value": 10}}
      ]
    }}
    '''
    
    circuit = pulsim.parse_netlist_string(netlist)
    options = pulsim.SimulationOptions()
    options.tstop = 30e-6
    options.dt = 10e-9
    
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    t = np.array(data['time']) * 1e6
    v_sw = np.array(data['signals']['V(sw)'])
    
    label = f'Dead-time = {dt*1e9:.0f} ns' if dt > 0 else 'No dead-time (shoot-through!)'
    color = 'r' if dt == 0 else 'b'
    
    axes[idx].plot(t, v_sw, color=color, linewidth=1)
    axes[idx].set_ylabel('V(sw) (V)')
    axes[idx].set_title(label)
    axes[idx].grid(True)
    axes[idx].set_ylim([-5, 55])

axes[-1].set_xlabel('Time (µs)')
plt.tight_layout()
plt.show()

print("Without dead-time: Shoot-through causes high currents and distorted waveform")
print("With dead-time: Clean switching with defined off-periods")

## 7. Synchronous Rectification

Synchronous rectification replaces diodes with MOSFETs for lower losses:

- **Diode**: Vf ≈ 0.5-0.7V forward drop
- **MOSFET**: Vds = I × Rds_on (can be < 0.1V)

Requires complementary PWM with dead-time.

In [ ]:
# Compare diode vs synchronous rectification
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Diode rectification
netlist_diode = '''
{
  "name": "Buck with Diode",
  "components": [
    {"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": 12},
    {"type": "V", "name": "Vpwm", "nodes": ["pwm", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": 100e3, "duty": 0.5}},
    {"type": "S", "name": "S1", "nodes": ["vin", "sw", "pwm", "0"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 5}},
    {"type": "D", "name": "D1", "nodes": ["0", "sw"],
     "params": {"ideal": false, "is": 1e-10, "n": 1.5, "rs": 0.01}},
    {"type": "L", "name": "L1", "nodes": ["sw", "out"], "value": 100e-6},
    {"type": "C", "name": "C1", "nodes": ["out", "0"], "value": 100e-6},
    {"type": "R", "name": "R", "nodes": ["out", "0"], "value": 2.5}
  ]
}
'''

# Synchronous rectification
netlist_sync = '''
{
  "name": "Buck with Sync Rect",
  "components": [
    {"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": 12},
    {"type": "V", "name": "Vpwm_h", "nodes": ["pwm_h", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": 100e3, "duty": 0.5, "dead_time": 50e-9}},
    {"type": "V", "name": "Vpwm_l", "nodes": ["pwm_l", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": 100e3, "duty": 0.5, "dead_time": 50e-9,
                  "complementary": true}},
    {"type": "S", "name": "S_h", "nodes": ["vin", "sw", "pwm_h", "0"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 5}},
    {"type": "S", "name": "S_l", "nodes": ["sw", "0", "pwm_l", "0"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 5}},
    {"type": "L", "name": "L1", "nodes": ["sw", "out"], "value": 100e-6},
    {"type": "C", "name": "C1", "nodes": ["out", "0"], "value": 100e-6},
    {"type": "R", "name": "R", "nodes": ["out", "0"], "value": 2.5}
  ]
}
'''

options = pulsim.SimulationOptions()
options.tstop = 200e-6
options.dt = 20e-9

for col, (netlist, title) in enumerate([(netlist_diode, 'Diode Rectification'),
                                         (netlist_sync, 'Synchronous Rectification')]):
    circuit = pulsim.parse_netlist_string(netlist)
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    t = np.array(data['time']) * 1e6
    v_sw = np.array(data['signals']['V(sw)'])
    v_out = np.array(data['signals']['V(out)'])
    
    axes[0, col].plot(t, v_sw, 'b-', linewidth=0.8)
    axes[0, col].set_ylabel('V(sw) (V)')
    axes[0, col].set_title(title)
    axes[0, col].grid(True)
    axes[0, col].set_ylim([-1.5, 14])
    
    axes[1, col].plot(t, v_out, 'r-', linewidth=1.5)
    axes[1, col].set_xlabel('Time (µs)')
    axes[1, col].set_ylabel('V(out) (V)')
    axes[1, col].grid(True)
    
    # Calculate average output
    v_avg = np.mean(v_out[-500:])
    axes[1, col].axhline(y=v_avg, color='g', linestyle='--', 
                         label=f'Avg: {v_avg:.2f}V')
    axes[1, col].legend()

plt.tight_layout()
plt.show()

print("Notice:")
print("  - Diode: V(sw) goes to ~-0.7V when freewheeling (diode drop)")
print("  - Sync Rect: V(sw) stays closer to 0V (MOSFET Rds_on drop)")
print("  - Lower losses with sync rect, especially at high currents")

## Summary

### Switch Parameters

```python
SwitchParams:
    ron           # On-resistance
    roff          # Off-resistance  
    vth           # Control threshold voltage
    initial_state # Starting state
```

### Event Detection

- Pulsim uses bisection to find exact switching moments
- Accurate timing regardless of timestep
- Switch events logged for analysis

### PWM Control

```python
PWMWaveform:
    frequency     # Switching frequency
    duty          # Duty cycle (0-1)
    dead_time     # Dead-time at both edges
    complementary # Invert for low-side
    phase         # Phase offset (0-1)
```

### Best Practices

1. **Dead-time**: Always use for half-bridges (100ns-1µs typical)
2. **Ron selection**: Match expected device Rds_on
3. **Timestep**: Set fine enough for dead-time resolution
4. **Initial state**: Consider startup conditions

**Next:** [Power Loss Calculation](11_power_losses.ipynb)